## Real-time Speech-to-Text and Response System using Hugging Face Transformers (HFT)


### Installing necessary packages and importing necessary modules

In [34]:
# Installing necessary packages
%pip install transformers
%pip install torch torchaudio
%pip install scipy
%pip install sounddevice 
%pip install gTTS
%pip install playsound
%pip install flask
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: torch in c:\users\ravik\appdata\local\programs\python\python310\lib\site-packages (2.5.1)




[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [109]:
# Importing necessary modules
from transformers import pipeline
from transformers import BlenderbotTokenizer,BlenderbotForConditionalGeneration
from gtts import gTTS
import sounddevice as sd
import torch
from playsound import playsound
import scipy.io.wavfile as wav
import numpy as np
from flask import Flask,send_file

### Capturing the audio input using sounddevice

In [110]:
# Callback function for audio input stream
def processing(data,frames,time,status):
    input_audio.append(data.copy()) # Appends the copy of the audio chunk to the list named input_audio

input_audio = []  # List to store chunks of the recorded audio

# Creating an audio input stream for recording
with sd.InputStream(samplerate=44100,channels=1,callback=processing): 
    print("Recording Started...")
    input("Press any key to stop recording")
    print("Recording Ended...")

audio = np.concatenate(input_audio,axis=0) # Combines all recorded audio chunks

# Saving the recording as a .wav file
wav.write("audio_record.wav",44100,audio)
print("Recording Done Sucessfully")

Recording Started...
Recording Ended...
Recording Done Sucessfully


### Converting the recorded speech into text using Whisper (HFT)

In [111]:
# Converting the recorded speech into text
transcriber = pipeline("automatic-speech-recognition",model="openai/whisper-small",generate_kwargs={"language": "en"}) # Creating a Pipeline
transcript = transcriber("audio_record.wav") # Transcribing the speech to text
resultant_text = transcript["text"] # Obtaining the transcribed text
print("\nTranscribed text:",resultant_text) # Displaying the transcribed text

Device set to use cpu



Transcribed text:  How was the weather today?


### Generating response based on the transcribed text using Facebook-Blenderbot (HFT)

In [112]:
# Generating a response based on the transcribed text

# Loading the BlenderBot model and tokenizer
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Function for the bot
def bot(user_input):
    inputs = tokenizer(user_input, return_tensors='pt') # Encoding the input
    reply_ids = model.generate(**inputs,do_sample=True,top_p=0.9) # Generating the response with top-p sampling method
    response = tokenizer.decode(reply_ids[0], skip_special_tokens=True) # Decoding the output
    return response

resultant_response = bot(resultant_text) # Obtaining the response text
print("Input Prompt:",resultant_text)
print("Bot response:", resultant_response) # Displaying the response text

Input Prompt:  How was the weather today?
Bot response:  It was a little chilly, but not too bad. It was nice to have a cool breeze.


### Converting the response to speech using gTTS (Google Text-to-Speech)

In [113]:
# Converting the response to speech
resultant_speech = gTTS(text=resultant_response,lang="en") # Using gTTS to convert the text to speech
resultant_speech.save("audio_response.mp3") # Saving the audio file
playsound(r"C:\Users\ravik\OneDrive\Desktop\Internship\audio_response.mp3") # Playing the audio file

### Displaying the responses in a web interface using Flask

In [114]:
# Using flask for web interface
app = Flask(__name__) # Creates a flask app

@app.route("/") # Creating the route for the webpage
def display(): # Function to display the contents 
    content = """
        <html>
            <body style="text-align:center; margin-top:150px;">
                <div style="display:flex;justify-content:center;align-items:center;gap:10px;">
                    <h2>Input Audio:</h2>
                    <audio controls style="width:300px;">
                        <source src="/audio1" type="audio/wav">
                    </audio>
                </div>
                <h2><br>Transcribed Text: {}</h2>
                <h2><br>Bot Response:{}</h2>
                <div style="display:flex;justify-content:center;align-items:center;gap:10px;margin-top:50px;">
                    <h2>Response Audio:</h2>
                    <audio controls style="width:300px;">
                        <source src="/audio2" type="audio/mp3">
                    </audio>
                </div>   
            </body>
        </html>""".format(resultant_text,resultant_response)     
    
    return content   

@app.route("/audio1") # Creating the route for the input audio
def play_input_audio(): # Function to display the input audio
    aud = send_file("audio_record.wav",mimetype="audio/wav")
    return aud

@app.route("/audio2") # Creating the route for the response audio
def play_response_audio(): # Function to display the response audio
    aud = send_file("audio_response.mp3",mimetype="audio/mp3")
    return aud

app.run() # Runs flask app
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jan/2025 23:51:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2025 23:51:38] "GET /audio1 HTTP/1.1" 206 -
127.0.0.1 - - [26/Jan/2025 23:51:38] "GET /audio2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2025 23:51:39] "GET /audio2 HTTP/1.1" 206 -
127.0.0.1 - - [26/Jan/2025 23:51:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2025 23:51:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2025 23:51:53] "GET /audio1 HTTP/1.1" 206 -
127.0.0.1 - - [26/Jan/2025 23:52:01] "GET /audio2 HTTP/1.1" 206 -
